In [1]:
# https://stackoverflow.com/questions/39299838/how-do-i-import-module-in-jupyter-notebook-directory-into-notebooks-in-lower-dir
# https://stackoverflow.com/questions/68572852/import-local-modules-in-jupyter-notebook
import os
import sys
# os.path.split(os.getcwd())[0]
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/models")
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools")
for nb_dir in sys.path:
    print(nb_dir)
    # sys.path.append(nb_dir)


/gpfs/home/dhuang/thesis/WavCaps
/home/dhuang/.conda/envs/fs/lib/python310.zip
/home/dhuang/.conda/envs/fs/lib/python3.10
/home/dhuang/.conda/envs/fs/lib/python3.10/lib-dynload

/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/huggingface_hub-0.14.1-py3.8.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/sentencepiece-0.1.95-py3.10-linux-x86_64.egg
/gpfs/home/dhuang/thesis/WavCaps/retrieval/models
/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools


In [1]:
import torch
import torch.nn as nn
from models.audio_encoder import AudioEncoder
from models.text_encoder import TextEncoder
import torch.nn.functional as F
import copy
from tools.losses import AudioTextContrastiveLoss, NTXent
from tools.utils import remove_grad
import ruamel.yaml as yaml
import librosa
import random
import numpy as np


In [2]:
config_file_path = './settings/dac_embedding.yaml'

with open(config_file_path, "r") as f:
        config = yaml.safe_load(f)

In [3]:
# Load audio signal file
# wav_file_path = '../../dac/audio_samples/at2_32khz_cvt.wav'
wav_file_path = '../../dac/audio_samples/at2_cvt.wav'
waveform, _ = librosa.load(wav_file_path, sr=config["audio_args"]["sr"], mono=True)
print('waveform shape before crop: ', waveform.shape)
if config["audio_args"]["max_length"] != 0:
            # if audio length is longer than max_length, we random crop it
            max_length = config["audio_args"]["max_length"] * config["audio_args"]["sr"]
            if waveform.shape[-1] > max_length:
                max_start = waveform.shape[-1] - max_length
                start = random.randint(0, max_start)
                waveform = waveform[start: start + max_length]
                
print('waveform shape: ', waveform.shape)
waveform_tensor = torch.tensor(waveform[None, :])
print('waveform_tensor shape: ', waveform_tensor.shape)


waveform shape before crop:  (240000,)
waveform shape:  (160000,)
waveform_tensor shape:  torch.Size([1, 160000])


In [4]:
from models.feature_extractor import AudioFeature, DACLatents
AF = AudioFeature(config["audio_args"])
DAC_Latents_Enc = DACLatents(config)

/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/torchlibrosa/stft.py:686: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,


In [5]:
audio_encoded = AF(waveform_tensor)
print('audio_encoded.shape: ', audio_encoded.shape)

audio_encoded.shape:  torch.Size([1, 1, 1001, 64])


In [6]:
bn0 = nn.BatchNorm2d(64)  #(self.config["audio_args"]["n_mels"])
# print('bn0.shape: ', bn0.shape)
print(bn0)

audio_encoded = audio_encoded.transpose(1, 3)
print('audio_encoded.shape: ', audio_encoded.shape)
# print(audio_encoded)

audio_encoded = bn0(audio_encoded)
print('audio_encoded.shape: ', audio_encoded.shape)
# print(audio_encoded)

audio_encoded = audio_encoded.transpose(1, 3)
print('audio_encoded.shape: ', audio_encoded.shape)
# print(audio_encoded)

# x = self.reshape_wav2img(x)


BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
audio_encoded.shape:  torch.Size([1, 64, 1001, 1])
audio_encoded.shape:  torch.Size([1, 64, 1001, 1])
audio_encoded.shape:  torch.Size([1, 1, 1001, 64])


In [5]:
lats = DAC_Latents_Enc(waveform_tensor)
lats.shape

torch.Size([1, 96, 500])

In [6]:
a,b,c = lats.shape
a,b,c

(1, 96, 500)

In [ ]:
"""HTSAT based on the Swin Transformer
Args:
    spec_size (int | tuple(int)): Input Spectrogram size. Default 256
    patch_size (int | tuple(int)): Patch size. Default: 4
    path_stride (iot | tuple(int)): Patch Stride for Frequency and Time Axis. Default: 4
    in_chans (int): Number of input image channels. Default: 1 (mono)
    num_classes (int): Number of classes for classification head. Default: 527
    embed_dim (int): Patch embedding dimension. Default: 96
    depths (tuple(int)): Depth of each HTSAT-Swin Transformer layer.
    num_heads (tuple(int)): Number of attention heads in different layers.
    window_size (int): Window size. Default: 8
    mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
    qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
    qk_scale (float): Override default qk scale of head_dim ** -0.5 if set. Default: None
    drop_rate (float): Dropout rate. Default: 0
    attn_drop_rate (float): Attention dropout rate. Default: 0
    drop_path_rate (float): Stochastic depth rate. Default: 0.1
    norm_layer (nn.Module): Normalization layer. Default: nn.LayerNorm.
    ape (bool): If True, add absolute position embedding to the patch embedding. Default: False
    patch_norm (bool): If True, add normalization after patch embedding. Default: True
    use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False
    config (module): The configuration Module from config.py
"""

# self.freq_ratio = self.spec_size // self.config["audio_args"]["n_mels"] = 256//64 = 4
# target_T = int(self.spec_size * self.freq_ratio) = 256*4 = 1024
# target_F = self.spec_size // self.freq_ratio = 256//4 = 64

# [B, C, T, F] = [B, 1, <=1024, <= 64]  ==>>  1024*64 = 65536

In [5]:
256//(256//96)

128

In [7]:
(7 - 7%3) // 3

2

In [7]:
256//64

4

In [15]:
256//4

64

In [16]:
1024*64

65536

In [18]:
audio_encoder = AudioEncoder(config)
# settings for projection layers
embed_size = config["embed_size"]
audio_width = audio_encoder.audio_width

In [29]:
with torch.no_grad():
    # audio_encoded = audio_encoder.audio_enc(np.ndarray([[0], waveform]) )
    audio_encoded = audio_encoder.audio_enc(waveform_tensor)
    # audio_encoded = audio_encoder.audio_enc(torch.randn(1, 1, 64, 1024))
    # audio_feats = audio_encoder(waveform)
    # audio_embeds = F.normalize(audio_proj(audio_feats), dim=-1)

print('audio_encoded.shape: ', audio_encoded.shape)

audio_encoded.shape:  torch.Size([1, 768])


In [5]:
from models.htsat import HTSAT_Swin_Transformer

htsat_audio_enc = HTSAT_Swin_Transformer(
                spec_size=256,
                patch_size=4,
                patch_stride=(4, 4),
                num_classes=527,
                embed_dim=96,
                depths=[2, 2, 6, 2],
                num_heads=[4, 8, 16, 32],
                window_size=8,
                config=config,
            )
audio_ckpt = torch.load("../../pretrained_models/audio_encoder/HTSAT.ckpt", map_location="cpu")["state_dict"]
for key in list(audio_ckpt.keys()):
    print("key contains: ", key)
    if key.startswith('sed_model') and ('spectrogram_extractor' not in key
                                        and 'logmel_extractor' not in key):
        v = audio_ckpt.pop(key)
        audio_ckpt[key[10:]] = v
htsat_audio_enc.load_state_dict(audio_ckpt, strict=False)
param_names = [n for n, p in htsat_audio_enc.named_parameters()]
for n in param_names:
    print(n, "\t", "Loaded" if n in audio_ckpt else "Unloaded")

key contains:  sed_model.spectrogram_extractor.stft.conv_real.weight
key contains:  sed_model.spectrogram_extractor.stft.conv_imag.weight
key contains:  sed_model.logmel_extractor.melW
key contains:  sed_model.bn0.weight
key contains:  sed_model.bn0.bias
key contains:  sed_model.bn0.running_mean
key contains:  sed_model.bn0.running_var
key contains:  sed_model.bn0.num_batches_tracked
key contains:  sed_model.patch_embed.proj.weight
key contains:  sed_model.patch_embed.proj.bias
key contains:  sed_model.patch_embed.norm.weight
key contains:  sed_model.patch_embed.norm.bias
key contains:  sed_model.layers.0.blocks.0.norm1.weight
key contains:  sed_model.layers.0.blocks.0.norm1.bias
key contains:  sed_model.layers.0.blocks.0.attn.relative_position_bias_table
key contains:  sed_model.layers.0.blocks.0.attn.relative_position_index
key contains:  sed_model.layers.0.blocks.0.attn.qkv.weight
key contains:  sed_model.layers.0.blocks.0.attn.qkv.bias
key contains:  sed_model.layers.0.blocks.0.att

In [3]:
from models.Codes_htsat import Codec_Swin_Transformer

htsat_audio_enc = Codec_Swin_Transformer(
                spec_size=256,
                patch_size=4,
                patch_stride=(4, 4),
                num_classes=527,
                embed_dim=96,
                depths=[2, 2, 6, 2],
                num_heads=[4, 8, 16, 32],
                window_size=8,
                config=config,
            )
audio_ckpt = torch.load("../../pretrained_models/audio_encoder/HTSAT.ckpt", map_location="cpu")["state_dict"]
for key in list(audio_ckpt.keys()):
    print("key contains: ", key)
    if key.startswith('sed_model') and ('spectrogram_extractor' not in key
                                        and 'logmel_extractor' not in key):
        v = audio_ckpt.pop(key)
        audio_ckpt[key[10:]] = v
htsat_audio_enc.load_state_dict(audio_ckpt, strict=False)
param_names = [n for n, p in htsat_audio_enc.named_parameters()]
for n in param_names:
    print(n, "\t", "Loaded" if n in audio_ckpt else "Unloaded")

/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


key contains:  sed_model.spectrogram_extractor.stft.conv_real.weight
key contains:  sed_model.spectrogram_extractor.stft.conv_imag.weight
key contains:  sed_model.logmel_extractor.melW
key contains:  sed_model.bn0.weight
key contains:  sed_model.bn0.bias
key contains:  sed_model.bn0.running_mean
key contains:  sed_model.bn0.running_var
key contains:  sed_model.bn0.num_batches_tracked
key contains:  sed_model.patch_embed.proj.weight
key contains:  sed_model.patch_embed.proj.bias
key contains:  sed_model.patch_embed.norm.weight
key contains:  sed_model.patch_embed.norm.bias
key contains:  sed_model.layers.0.blocks.0.norm1.weight
key contains:  sed_model.layers.0.blocks.0.norm1.bias
key contains:  sed_model.layers.0.blocks.0.attn.relative_position_bias_table
key contains:  sed_model.layers.0.blocks.0.attn.relative_position_index
key contains:  sed_model.layers.0.blocks.0.attn.qkv.weight
key contains:  sed_model.layers.0.blocks.0.attn.qkv.bias
key contains:  sed_model.layers.0.blocks.0.att